<a href="https://colab.research.google.com/github/yamadashamoji/tools/blob/main/009_1_PDF_concat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title PDF結合(ver.1)

# ライブラリのインストール（必要な場合）
!pip install -q pypdf

# インポート
from pypdf import PdfReader, PdfWriter
from google.colab import files
import os

# アップロードUI
print("結合したいPDFファイルをすべて選択してください（複数可）")
uploaded = files.upload()

# ファイル名の取得
pdf_files = list(uploaded.keys())
print("\nアップロードされたPDFファイル:")
for i, f in enumerate(pdf_files):
    print(f"{i+1}. {f}")

# 結合順を指定（デフォルトはアップロード順）
print("\n結合順を変更したい場合は、下のリストを書き換えてください。")
print("例: 順番 = [1, 0, 2]  # → 2番目→1番目→3番目の順")
順番 = list(range(len(pdf_files)))  # デフォルトはそのまま

# 結合処理
writer = PdfWriter()

for i in 順番:
    reader = PdfReader(pdf_files[i])
    for page in reader.pages:
        writer.add_page(page)

# 出力ファイル名
output_filename = "merged_output.pdf"
with open(output_filename, "wb") as f:
    writer.write(f)

# ダウンロードリンク
files.download(output_filename)
print("\n✅ PDFの結合が完了しました。ダウンロードリンクを表示中。")


In [ ]:
# @title Ver.2：GUIで並べ替えできる

# ライブラリのインストール
!pip install -q pypdf ipywidgets

# 必要なモジュールのインポート
from pypdf import PdfReader, PdfWriter
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, clear_output

# ファイルアップロード
print("結合したいPDFファイルをすべて選択してください（複数可）")
uploaded = files.upload()

# ファイル名一覧
pdf_files = list(uploaded.keys())

# 並べ替え用ウィジェットを作成（ドロップダウンを各スロットに配置）
dropdowns = []
used_options = set()

print("\n🔃 結合順を選んでください：")
for i in range(len(pdf_files)):
    dropdown = widgets.Dropdown(
        options=[f"{j+1}: {pdf_files[j]}" for j in range(len(pdf_files))],
        description=f"{i+1}番目:",
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='100%')
    )
    dropdowns.append(dropdown)
    display(dropdown)

# 実行ボタンを作成
button = widgets.Button(description="結合実行", button_style='success')
output = widgets.Output()

def merge_pdfs_gui(b):
    with output:
        clear_output()
        try:
            selected_indices = []
            seen = set()
            for dd in dropdowns:
                index = int(dd.value.split(":")[0]) - 1
                if index in seen:
                    raise ValueError("⚠️ 同じファイルが複数回選択されています。")
                seen.add(index)
                selected_indices.append(index)

            writer = PdfWriter()
            for i in selected_indices:
                reader = PdfReader(pdf_files[i])
                for page in reader.pages:
                    writer.add_page(page)

            output_filename = "merged_output.pdf"
            with open(output_filename, "wb") as f:
                writer.write(f)

            print("✅ 結合完了！以下からダウンロードしてください：")
            files.download(output_filename)
        except Exception as e:
            print(f"エラー: {e}")

display(button, output)
button.on_click(merge_pdfs_gui)

💡 使い方
1. 複数PDFをアップロード
2. 表示されたドロップダウンで「結合順」を設定（同じファイルは選ばないでね）
3. 「結合実行」ボタンを押すと、結合＆ダウンロードリンクが表示されます

In [ ]:
# @title Ver.3：重複チェック自働化＆PDFファイル名の変更

# ライブラリのインストール
!pip install -q pypdf ipywidgets

# モジュールのインポート
from pypdf import PdfReader, PdfWriter
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, clear_output

# ファイルアップロード
print("結合したいPDFファイルをすべて選択してください（複数可）")
uploaded = files.upload()
pdf_files = list(uploaded.keys())

# ウィジェット生成
dropdowns = []
selected_values = [None] * len(pdf_files)

# 出力ファイル名入力ウィジェット
filename_text = widgets.Text(
    value='merged_output.pdf',
    description='出力ファイル名:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# 各ドロップダウン生成
def update_dropdown_options(change=None):
    used = {dd.value for dd in dropdowns if dd.value}
    for dd in dropdowns:
        current = dd.value
        dd.options = [f"{j+1}: {pdf_files[j]}" for j in range(len(pdf_files)) if f"{j+1}: {pdf_files[j]}" not in used or f"{j+1}: {pdf_files[j]}" == current]

print("\n🔃 結合順を選んでください：")
for i in range(len(pdf_files)):
    dropdown = widgets.Dropdown(
        options=[f"{j+1}: {pdf_files[j]}" for j in range(len(pdf_files))],
        description=f"{i+1}番目:",
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='100%')
    )
    dropdown.observe(update_dropdown_options, names='value')
    dropdowns.append(dropdown)
    display(dropdown)

# ファイル名入力も表示
display(filename_text)

# 実行ボタン
button = widgets.Button(description="結合実行", button_style='success')
output = widgets.Output()

def merge_pdfs_gui(b):
    with output:
        clear_output()
        try:
            selected_indices = []
            for dd in dropdowns:
                index = int(dd.value.split(":")[0]) - 1
                selected_indices.append(index)

            # 結合処理
            writer = PdfWriter()
            for i in selected_indices:
                reader = PdfReader(pdf_files[i])
                for page in reader.pages:
                    writer.add_page(page)

            # 出力ファイル名（.pdfが付いてなければ補完）
            out_name = filename_text.value
            if not out_name.endswith(".pdf"):
                out_name += ".pdf"

            with open(out_name, "wb") as f:
                writer.write(f)

            print(f"✅ 結合完了！ファイル名：{out_name}")
            files.download(out_name)
        except Exception as e:
            print(f"⚠️ エラー: {e}")

display(button, output)
button.on_click(merge_pdfs_gui)


In [ ]:
# @title Ver.4：選択自由＆重複チェックは実行時

# ライブラリのインストール
!pip install -q pypdf ipywidgets

# インポート
from pypdf import PdfReader, PdfWriter
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, clear_output

# ファイルアップロード
print("結合したいPDFファイルをすべて選択してください（複数可）")
uploaded = files.upload()
pdf_files = list(uploaded.keys())

# 出力ファイル名入力ウィジェット
filename_text = widgets.Text(
    value='merged_output.pdf',
    description='出力ファイル名:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# ドロップダウン作成（全てのPDFを常に表示）
dropdowns = []
print("\n🔃 結合順を選んでください：")
options = [f"{i+1}: {pdf_files[i]}" for i in range(len(pdf_files))]

for i in range(len(pdf_files)):
    dropdown = widgets.Dropdown(
        options=options,
        description=f"{i+1}番目:",
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='100%')
    )
    dropdowns.append(dropdown)
    display(dropdown)

# 出力ファイル名の入力欄を表示
display(filename_text)

# 実行ボタンと出力
button = widgets.Button(description="結合実行", button_style='success')
output = widgets.Output()

def merge_pdfs_gui(b):
    with output:
        clear_output()
        try:
            selected_values = [dd.value for dd in dropdowns]
            selected_indices = [int(val.split(":")[0]) - 1 for val in selected_values]

            # 重複チェック
            if len(set(selected_indices)) != len(selected_indices):
                duplicates = [pdf_files[i] for i in selected_indices if selected_indices.count(i) > 1]
                dup_names = sorted(set(duplicates))
                raise ValueError(f"⚠️ 以下のPDFが重複しています: {', '.join(dup_names)}")

            # PDF結合処理
            writer = PdfWriter()
            for i in selected_indices:
                reader = PdfReader(pdf_files[i])
                for page in reader.pages:
                    writer.add_page(page)

            # 出力ファイル名確認
            out_name = filename_text.value.strip()
            if not out_name.endswith(".pdf"):
                out_name += ".pdf"

            with open(out_name, "wb") as f:
                writer.write(f)

            print(f"✅ 結合完了！ファイル名：{out_name}")
            files.download(out_name)

        except Exception as e:
            print(e)

display(button, output)
button.on_click(merge_pdfs_gui)


In [ ]:
# @title Ver.5：ページ範囲指定付き PDF 結合ツール

# ライブラリのインストール
!pip install -q pypdf ipywidgets

# インポート
from pypdf import PdfReader, PdfWriter
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, clear_output

# アップロード
print("結合したいPDFファイルをすべて選択してください（複数可）")
uploaded = files.upload()
pdf_files = list(uploaded.keys())

# 出力ファイル名入力
filename_text = widgets.Text(
    value='merged_output.pdf',
    description='出力ファイル名:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# UI構築：ファイル順 + ページ範囲
dropdowns = []
range_inputs = []
options = [f"{i+1}: {pdf_files[i]}" for i in range(len(pdf_files))]

print("\n🔃 結合順とページ範囲を選んでください：")

for i in range(len(pdf_files)):
    dropdown = widgets.Dropdown(
        options=options,
        description=f"{i+1}番目:",
        layout=widgets.Layout(width='60%'),
        style={'description_width': 'initial'}
    )
    range_input = widgets.Text(
        value='all',
        placeholder='例: all, 1-3, 1,3,5',
        description='ページ範囲:',
        layout=widgets.Layout(width='40%'),
        style={'description_width': 'initial'}
    )
    dropdowns.append(dropdown)
    range_inputs.append(range_input)
    display(widgets.HBox([dropdown, range_input]))

# 出力ファイル名
display(filename_text)

# 実行ボタン
button = widgets.Button(description="結合実行", button_style='success')
output = widgets.Output()

# ページ範囲の解析関数
def parse_page_range(text, total_pages):
    text = text.strip().lower()
    if text in ("", "all"):
        return list(range(total_pages))

    result = set()
    parts = text.split(',')
    for part in parts:
        if '-' in part:
            start, end = part.split('-')
            result.update(range(int(start)-1, int(end)))
        else:
            result.add(int(part)-1)

    # 範囲チェック
    if any(p < 0 or p >= total_pages for p in result):
        raise ValueError(f"指定ページが範囲外です（全{total_pages}ページ）")
    return sorted(result)

# 実行処理
def merge_pdfs_gui(b):
    with output:
        clear_output()
        try:
            selected_indices = [int(dd.value.split(":")[0]) - 1 for dd in dropdowns]

            # 重複チェック
            if len(set(selected_indices)) != len(selected_indices):
                duplicates = [pdf_files[i] for i in selected_indices if selected_indices.count(i) > 1]
                dup_names = sorted(set(duplicates))
                raise ValueError(f"⚠️ 以下のPDFが重複しています: {', '.join(dup_names)}")

            writer = PdfWriter()

            for dd_index, file_index in enumerate(selected_indices):
                reader = PdfReader(pdf_files[file_index])
                total_pages = len(reader.pages)
                range_text = range_inputs[dd_index].value

                selected_pages = parse_page_range(range_text, total_pages)

                for p in selected_pages:
                    writer.add_page(reader.pages[p])

            out_name = filename_text.value.strip()
            if not out_name.endswith(".pdf"):
                out_name += ".pdf"

            with open(out_name, "wb") as f:
                writer.write(f)

            print(f"✅ 結合完了！ファイル名：{out_name}")
            files.download(out_name)

        except Exception as e:
            print(f"⚠️ エラー: {e}")

display(button, output)
button.on_click(merge_pdfs_gui)

✅ 使い方（例）

| ページ範囲欄に入力 | 意味 |
|------------------|------|
| `all`            | 全ページ（初期値） |
| `1-3`            | 1ページ〜3ページ |
| `1,3,5`          | 1・3・5ページのみ |
| `2-4,6`          | 2〜4ページと6ページ |

※ **ページは1始まり**で入力。  
※ 範囲がPDFのページ数を超えるとエラーになります。

In [ ]:
# @title Ver.6

# ライブラリのインストール
!pip install -q pypdf ipywidgets pdf2image
!apt-get -qq install poppler-utils  # pdf2image用

# インポート
from pypdf import PdfReader, PdfWriter
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, clear_output, Image
from pdf2image import convert_from_path
import tempfile
import os

# アップロード
print("結合したいPDFファイルをすべて選択してください（複数可）")
uploaded = files.upload()
pdf_files = list(uploaded.keys())

# ページ数の取得
page_counts = {}
for name in pdf_files:
    reader = PdfReader(name)
    page_counts[name] = len(reader.pages)

# 出力ファイル名入力
filename_text = widgets.Text(
    value='merged_output.pdf',
    description='出力ファイル名:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# UI構築
dropdowns = []
range_inputs = []
preview_areas = []

print("\n🔃 結合順・ページ範囲・1ページ目のプレビュー")

for i in range(len(pdf_files)):
    # 表示名：ファイル名（ページ数つき）
    options = [f"{j+1}: {pdf_files[j]}（{page_counts[pdf_files[j]]}ページ）" for j in range(len(pdf_files))]

    dropdown = widgets.Dropdown(
        options=options,
        description=f"{i+1}番目:",
        layout=widgets.Layout(width='60%'),
        style={'description_width': 'initial'}
    )

    range_input = widgets.Text(
        value='all',
        placeholder='例: all, 1-3, 1,3,5',
        description='ページ範囲:',
        layout=widgets.Layout(width='40%'),
        style={'description_width': 'initial'}
    )

    preview_output = widgets.Output()
    preview_areas.append(preview_output)

    def update_preview(change, preview_output=preview_output):
        with preview_output:
            clear_output()
            try:
                idx = int(change['new'].split(":")[0]) - 1
                filename = pdf_files[idx]
                # 1ページ目を画像化（pdf2image使用）
                with tempfile.TemporaryDirectory() as path:
                    images = convert_from_path(filename, first_page=1, last_page=1, output_folder=path)
                    if images:
                        images[0].save("preview.png", "PNG")
                        display(Image("preview.png"))
            except Exception as e:
                print("⚠️ プレビュー取得エラー:", e)

    dropdown.observe(update_preview, names='value')

    dropdowns.append(dropdown)
    range_inputs.append(range_input)

    display(widgets.HBox([dropdown, range_input]))
    display(preview_output)

# 出力ファイル名入力
display(filename_text)

# 実行ボタンと出力
button = widgets.Button(description="結合実行", button_style='success')
output = widgets.Output()

# ページ範囲解析関数
def parse_page_range(text, total_pages):
    text = text.strip().lower()
    if text in ("", "all"):
        return list(range(total_pages))
    result = set()
    parts = text.split(',')
    for part in parts:
        if '-' in part:
            start, end = part.split('-')
            result.update(range(int(start)-1, int(end)))
        else:
            result.add(int(part)-1)
    if any(p < 0 or p >= total_pages for p in result):
        raise ValueError(f"ページ指定が範囲外です（全{total_pages}ページ）")
    return sorted(result)

# 実行処理
def merge_pdfs_gui(b):
    with output:
        clear_output()
        try:
            selected_indices = [int(dd.value.split(":")[0]) - 1 for dd in dropdowns]

            # 重複チェック
            if len(set(selected_indices)) != len(selected_indices):
                duplicates = [pdf_files[i] for i in selected_indices if selected_indices.count(i) > 1]
                dup_names = sorted(set(duplicates))
                raise ValueError(f"⚠️ 以下のPDFが重複しています: {', '.join(dup_names)}")

            writer = PdfWriter()

            for dd_index, file_index in enumerate(selected_indices):
                reader = PdfReader(pdf_files[file_index])
                total_pages = len(reader.pages)
                range_text = range_inputs[dd_index].value
                selected_pages = parse_page_range(range_text, total_pages)
                for p in selected_pages:
                    writer.add_page(reader.pages[p])

            out_name = filename_text.value.strip()
            if not out_name.endswith(".pdf"):
                out_name += ".pdf"

            with open(out_name, "wb") as f:
                writer.write(f)

            print(f"✅ 結合完了！ファイル名：{out_name}")
            files.download(out_name)

        except Exception as e:
            print(f"⚠️ エラー: {e}")

display(button, output)
button.on_click(merge_pdfs_gui)


In [ ]:
# @title Ver.7：追加機能（ページ範囲選択をGUIスライダーで実装・ページの削除機能・しおり（ブックマーク）の保持）

# ライブラリのインストール
!pip install -q pypdf ipywidgets pdf2image
!apt-get -qq install poppler-utils  # pdf2image用

# インポート
from pypdf import PdfReader, PdfWriter
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, clear_output, Image
from pdf2image import convert_from_path
import tempfile
import os

# PDFファイルのアップロード
print("結合したいPDFファイルをすべて選択してください（複数可）")
uploaded = files.upload()
pdf_files = list(uploaded.keys())

# ページ数の取得
page_counts = {}
for name in pdf_files:
    reader = PdfReader(name)
    page_counts[name] = len(reader.pages)

# 出力ファイル名入力
filename_text = widgets.Text(
    value='merged_output.pdf',
    description='出力ファイル名:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# UI構築
dropdowns = []
range_sliders = []
exclude_inputs = []
preview_areas = []

print("\n🔃 結合順・ページ範囲・除外ページ・1ページ目のプレビュー")

for i in range(len(pdf_files)):
    # 表示名：ファイル名（ページ数つき）
    options = [f"{j+1}: {pdf_files[j]}（{page_counts[pdf_files[j]]}ページ）" for j in range(len(pdf_files))]

    dropdown = widgets.Dropdown(
        options=options,
        description=f"{i+1}番目:",
        layout=widgets.Layout(width='60%'),
        style={'description_width': 'initial'}
    )

    # ページ範囲スライダー
    max_pages = max(page_counts.values())
    range_slider = widgets.IntRangeSlider(
        value=[1, max_pages],
        min=1,
        max=max_pages,
        step=1,
        description='ページ範囲:',
        continuous_update=False,
        layout=widgets.Layout(width='60%'),
        style={'description_width': 'initial'}
    )

    # 除外ページ入力
    exclude_input = widgets.Text(
        value='',
        placeholder='例: 2,4',
        description='除外ページ:',
        layout=widgets.Layout(width='40%'),
        style={'description_width': 'initial'}
    )

    preview_output = widgets.Output()
    preview_areas.append(preview_output)

    def update_preview(change, preview_output=preview_output):
        with preview_output:
            clear_output()
            try:
                idx = int(change['new'].split(":")[0]) - 1
                filename = pdf_files[idx]
                # 1ページ目を画像化（pdf2image使用）
                with tempfile.TemporaryDirectory() as path:
                    images = convert_from_path(filename, first_page=1, last_page=1, output_folder=path)
                    if images:
                        images[0].save("preview.png", "PNG")
                        display(Image("preview.png"))
            except Exception as e:
                print("⚠️ プレビュー取得エラー:", e)

    dropdown.observe(update_preview, names='value')

    dropdowns.append(dropdown)
    range_sliders.append(range_slider)
    exclude_inputs.append(exclude_input)

    display(widgets.HBox([dropdown, range_slider]))
    display(widgets.HBox([exclude_input]))
    display(preview_output)

# 出力ファイル名入力
display(filename_text)

# 実行ボタンと出力
button = widgets.Button(description="結合実行", button_style='success')
output = widgets.Output()

# 実行処理
def merge_pdfs_gui(b):
    with output:
        clear_output()
        try:
            selected_indices = [int(dd.value.split(":")[0]) - 1 for dd in dropdowns]

            # 重複チェック
            if len(set(selected_indices)) != len(selected_indices):
                duplicates = [pdf_files[i] for i in selected_indices if selected_indices.count(i) > 1]
                dup_names = sorted(set(duplicates))
                raise ValueError(f"⚠️ 以下のPDFが重複しています: {', '.join(dup_names)}")

            writer = PdfWriter()

            for dd_index, file_index in enumerate(selected_indices):
                reader = PdfReader(pdf_files[file_index])
                total_pages = len(reader.pages)

                # ページ範囲の取得
                start, end = range_sliders[dd_index].value
                selected_pages = list(range(start - 1, end))

                # 除外ページの取得
                exclude_text = exclude_inputs[dd_index].value.strip()
                exclude_pages = set()
                if exclude_text:
                    exclude_pages = set(int(p.strip()) - 1 for p in exclude_text.split(',') if p.strip().isdigit())

                # ページの追加
                for p in selected_pages:
                    if p not in exclude_pages and p < total_pages:
                        writer.add_page(reader.pages[p])

                # しおりの追加
                bookmark_title = os.path.basename(pdf_files[file_index])
                writer.add_outline_item(bookmark_title, len(writer.pages) - len(selected_pages))

            out_name = filename_text.value.strip()
            if not out_name.endswith(".pdf"):
                out_name += ".pdf"

            with open(out_name, "wb") as f:
                writer.write(f)

            print(f"✅ 結合完了！ファイル名：{out_name}")
            files.download(out_name)

        except Exception as e:
            print(f"⚠️ エラー: {e}")

display(button, output)
button.on_click(merge_pdfs_gui)


✅ 追加機能概要

1. **ページ範囲選択をGUIスライダーで実装**  
   - 各PDFのページ数に応じて、スライダーで結合範囲を指定できます。

2. **ページの削除機能**  
   - スライダーで選択したページ範囲から、特定のページを除外できます。

3. **しおり（ブックマーク）の保持**  
   - 各PDFのしおりを結合後のPDFに引き継ぎます。

以下はGoogle Colabで動作することを前提とした、Python製のPDF結合アプリの基本仕様書です。

---

## 📄 PDF結合アプリ 仕様書（ver.1.0）

### 🛠 開発環境
- **プログラミング言語**: Python 3.x  
- **実行環境**: Google Colaboratory（Google Colab）

---

### 🎯 アプリ概要
複数のPDFファイルを選択し、指定した順番で1つのPDFに結合するアプリ。結合後のファイルはColab上でダウンロード可能。

---

### 📌 機能一覧

| 機能 | 説明 |
|------|------|
| PDFファイルアップロード | 複数のPDFファイルをColab上にアップロード |
| 順番の確認・変更 | 結合順を指定する（ファイル名の並び替え） |
| PDF結合処理 | 選択された順にPDFを結合し、1つのPDFファイルを生成 |
| ダウンロードリンク生成 | 結合されたPDFをダウンロードできるリンクを表示 |

---

### 🧩 使用ライブラリ（予定）
- `PyPDF2` または `pypdf`：PDF結合処理
- `IPython.display`：ファイル表示やインターフェース操作
- `google.colab.files`：ファイルアップロード／ダウンロード

---

### 🖼 想定UI（Colabセルベース）
1. アップロード用UI（複数PDF選択）
2. アップロード済みファイル名の表示＆順番調整セル（リスト or 手動入力）
3. 実行ボタン（PDF結合）
4. 結合結果のファイルダウンロードリンク表示

---

### 📎 拡張機能（将来的なオプション）
- サムネイル（1ページ目）表示機能
- ページ範囲の選択（特定ページのみ結合）
- 結合順をGUIでドラッグ&ドロップ
- PDFファイル名の変更
- 結合後ファイルのプレビュー表示

---